# Regression for SMP

In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

import numpy as np                      # linear algebra
import pandas as pd                     # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn import preprocessing;
from sklearn import cross_validation;
from sklearn import linear_model;

Dataset consists of following files:

• prices.csv: raw, as-is daily prices. Most of data spans from 2010 to the end 2016, for companies new on stock market date range is shorter. There have been approx. 140 stock splits in that time, this set doesn't account for that.

• prices-split-adjusted.csv: same as prices, but there have been added adjustments for splits.

• securities.csv: general description of each company with division on sectors

• fundamentals.csv: metrics extracted from annual SEC 10K fillings (2012-2016), should be enough to derive most of popular fundamental indicators.

In [45]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
a = os.listdir("/Users/rounakbose/Desktop/SMP_Code")
print (a)

['prices-split-adjusted.csv', '.DS_Store', 'fundamentals.csv', 'SMP_Regression.ipynb', 'prices.csv', 'securities.csv']


In [46]:
def prepare_data(df, forecast_col, forecast_out, test_size):
    
    label = df[forecast_col].shift(-forecast_out);       #creating new column called label with the last 5 rows are nan
    X = np.array(df[[forecast_col]]);                           #creating the feature array
    X = preprocessing.scale(X)                                 #processing the feature array
    X_lately = X[-forecast_out:]                                 #creating the column i want to use later in the predicting method
    X = X[:-forecast_out]                                           # X that will contain the training and testing
    label.dropna(inplace=True);                                #dropping na values
    y = np.array(label)                                               # assigning Y
    
    #cross validation 
    X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, y, test_size=test_size) 

    response = [X_train, X_test, Y_train, Y_test, X_lately];
    return response;

In [47]:
#loading the csv file
df = pd.read_csv("/Users/rounakbose/Desktop/SMP_Code/prices.csv") 

#choosing stock symbol
df=df[df.symbol=='GOOG']

In [48]:
forecast_col = 'close'                       #choosing which column to forecast
forecast_out = 10                             #how far to forecast 
test_size = 0.2;                                 #the size of my test set

#calling the method where the cross validation and data preperation is in
X_train, X_test, Y_train, Y_test , X_lately =prepare_data(df,forecast_col,forecast_out,test_size); 

In [49]:
learner = linear_model.LinearRegression();           #initializing linear regression model

learner.fit(X_train,Y_train);                                     #training the linear regression model
score=learner.score(X_test,Y_test);                      #testing the linear regression model

forecast= learner.predict(X_lately);                       #set that will contain the forecasted data

In [50]:
response={};                                                          #creating json object
response['test_score']=score; 

print(response, '\n');

response['forecast_set']=forecast;

print(response);

{'test_score': 0.9069184892214378} 

{'test_score': 0.9069184892214378, 'forecast_set': array([782.43807629, 785.59613057, 787.65811276, 785.93049742,
       782.86535987, 781.61140104, 783.13470099, 777.09728689,
       774.99811516, 764.80884477])}
